In [1]:
import json
import pandas as pd
import geopandas as gp
import matplotlib
import gtfs_kit as kit

In [2]:
feed = kit.feed.read_feed('../data/mmt_gtfs.zip', dist_units='km')
kit.validators.validate(feed, as_df=True, include_warnings=False)
feed = feed.drop_invalid_columns().clean()
week = feed.get_week(1)
feed.describe()

agency: dropping invalid column checkin_duration
agency: dropping invalid column bike_policy_url
calendar: dropping invalid column service_name
fare_attributes: dropping invalid column agency_id
feed_info: dropping invalid column feed_contact_email
feed_info: dropping invalid column feed_contact_url
routes: dropping invalid column service_id
routes: dropping invalid column route_service_name
routes: dropping invalid column bikes_allowed
shapes: dropping invalid column shape_code
stops: dropping invalid column agency_id
stops: dropping invalid column jurisdiction_id
stops: dropping invalid column relative_position
stops: dropping invalid column cardinal_direction
stops: dropping invalid column primary_street
stops: dropping invalid column address_range
stops: dropping invalid column cross_location
trips: dropping invalid column route_short_name
trips: dropping invalid column direction_name
trips: dropping invalid column shape_code
trips: dropping invalid column trip_type
trips: dropping

/home/charles/.local/lib/python3.8/site-packages/gtfs_kit/cleaners.py:46: FutureWarning: The default value of regex will change from True to False in a future version.
  f[column] = f[column].str.strip().str.replace(r"\s+", "_")


,indicator,value
0,agencies,[Metro Transit-City of Madison]
1,timezone,America/Chicago
2,start_date,20211205
3,end_date,20220514
4,num_routes,94
5,num_trips,13675
6,num_stops,1749
7,num_shapes,300
8,sample_date,20211209
9,num_routes_active_on_sample_date,43


In [3]:
routes = feed.get_routes(date=week[0])
trip_stats = feed.compute_trip_stats()
fts = feed.compute_feed_time_series(trip_stats, dates=week, freq="2H")
feed_stats = feed.compute_feed_stats(trip_stats, week)
rts = feed.compute_route_time_series(trip_stats, week, freq='12H')

/home/charles/.local/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [8]:
inds = ['service_distance', 'service_duration']
rids = ['9500', '9502']

simple_rts = rts.loc[:, (inds, )]

In [28]:
simple_rts['service_distance'].describe()

route_id,9899,9901,9902,9903,9904,9905,9907,9908,9909,9911,...,9950,9951,9953,9954,9955,9956,9957,9958,9959,9960
count,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,10.000000,10.00000,14.000000,10.000000,...,4.000000,10.000000,10.000000,10.000000,10.000000,4.000000,14.000000,14.000000,14.000000,10.000000
mean,376.196829,246.603371,208.253079,491.816507,336.985136,151.071629,46.488800,48.89085,96.063186,305.901100,...,111.174400,254.805300,142.978350,117.985550,108.437350,137.126450,350.431829,34.131793,33.031671,101.142950
std,114.182452,74.203005,61.807583,142.669205,94.152051,56.805963,3.964231,2.13596,27.738369,56.845484,...,48.139927,82.112625,6.011015,34.994134,0.025456,51.027545,157.506999,14.578707,11.782983,34.058427
min,236.525229,112.579800,96.365933,322.613844,217.022817,59.703820,42.728000,46.86450,65.617800,251.972739,...,69.484000,176.906424,137.275800,84.787200,108.413200,92.935300,65.736339,17.860333,19.581231,68.832289
25%,279.212429,213.901620,181.806408,368.104625,259.380675,125.569898,42.728000,46.86450,71.226100,251.972739,...,69.484000,176.906424,137.275800,84.787200,108.413200,92.935300,237.877206,17.860333,19.581231,68.832289
50%,382.295000,236.417580,199.380533,498.243150,342.578250,133.337489,46.488800,48.89085,92.958550,305.901100,...,111.174400,254.805300,142.978350,117.985550,108.437350,137.126450,365.837839,37.263150,36.062100,101.142950
75%,485.377571,326.481420,274.802367,628.381675,425.775825,217.107102,50.249600,50.91720,125.627300,359.829461,...,152.864800,332.704176,148.680900,151.183900,108.461500,181.317600,504.706961,47.898167,44.057769,133.453611
max,485.377571,326.481420,274.802367,628.885956,428.981883,217.107102,50.249600,50.91720,125.627300,359.829461,...,152.864800,332.704176,148.680900,151.183900,108.461500,181.317600,504.706961,47.898167,44.057769,133.453611


In [26]:
simple_rts['service_distance'].head()

route_id,9899,9901,9902,9903,9904,9905,9907,9908,9909,9911,...,9950,9951,9953,9954,9955,9956,9957,9958,9959,9960
datetime,,,,,,,,,,,,,,,,,,,,,
2021-12-06 00:00:00,279.212429,236.41758,199.380533,368.104625,259.380675,125.569898,42.7280,50.9172,71.2261,251.972739,...,NaN,176.906424,137.2758,84.7872,108.4615,NaN,365.837839,17.860333,19.581231,68.832289
2021-12-06 12:00:00,485.377571,326.48142,274.802367,628.381675,425.775825,217.107102,50.2496,46.8645,125.6273,359.829461,...,NaN,332.704176,148.6809,151.1839,108.4132,NaN,504.706961,47.898167,44.057769,133.453611
2021-12-07 00:00:00,279.212429,236.41758,199.380533,368.104625,259.380675,125.569898,42.7280,50.9172,71.2261,251.972739,...,NaN,176.906424,137.2758,84.7872,108.4615,NaN,365.837839,17.860333,19.581231,68.832289
2021-12-07 12:00:00,485.377571,326.48142,274.802367,628.381675,425.775825,217.107102,50.2496,46.8645,125.6273,359.829461,...,NaN,332.704176,148.6809,151.1839,108.4132,NaN,504.706961,47.898167,44.057769,133.453611
2021-12-08 00:00:00,279.212429,236.41758,199.380533,368.104625,259.380675,125.569898,42.7280,50.9172,71.2261,251.972739,...,NaN,176.906424,137.2758,84.7872,108.4615,NaN,365.837839,17.860333,19.581231,68.832289


In [21]:
simple_rts['service_distance']

route_id,9899,9901,9902,9903,9904,9905,9907,9908,9909,9911,...,9950,9951,9953,9954,9955,9956,9957,9958,9959,9960
datetime,,,,,,,,,,,,,,,,,,,,,
2021-12-06 00:00:00,279.212429,236.41758,199.380533,368.104625,259.380675,125.569898,42.7280,50.9172,71.2261,251.972739,...,NaN,176.906424,137.2758,84.7872,108.4615,NaN,365.837839,17.860333,19.581231,68.832289
2021-12-06 12:00:00,485.377571,326.48142,274.802367,628.381675,425.775825,217.107102,50.2496,46.8645,125.6273,359.829461,...,NaN,332.704176,148.6809,151.1839,108.4132,NaN,504.706961,47.898167,44.057769,133.453611
2021-12-07 00:00:00,279.212429,236.41758,199.380533,368.104625,259.380675,125.569898,42.7280,50.9172,71.2261,251.972739,...,NaN,176.906424,137.2758,84.7872,108.4615,NaN,365.837839,17.860333,19.581231,68.832289
2021-12-07 12:00:00,485.377571,326.48142,274.802367,628.381675,425.775825,217.107102,50.2496,46.8645,125.6273,359.829461,...,NaN,332.704176,148.6809,151.1839,108.4132,NaN,504.706961,47.898167,44.057769,133.453611
2021-12-08 00:00:00,279.212429,236.41758,199.380533,368.104625,259.380675,125.569898,42.7280,50.9172,71.2261,251.972739,...,NaN,176.906424,137.2758,84.7872,108.4615,NaN,365.837839,17.860333,19.581231,68.832289
2021-12-08 12:00:00,485.377571,326.48142,274.802367,628.381675,425.775825,217.107102,50.2496,46.8645,125.6273,359.829461,...,NaN,332.704176,148.6809,151.1839,108.4132,NaN,504.706961,47.898167,44.057769,133.453611
2021-12-09 00:00:00,279.212429,236.41758,199.380533,368.104625,259.380675,125.569898,42.7280,50.9172,71.2261,251.972739,...,NaN,176.906424,137.2758,84.7872,108.4615,NaN,365.837839,17.860333,19.581231,68.832289
2021-12-09 12:00:00,485.377571,326.48142,274.802367,628.381675,425.775825,217.107102,50.2496,46.8645,125.6273,359.829461,...,NaN,332.704176,148.6809,151.1839,108.4132,NaN,504.706961,47.898167,44.057769,133.453611
2021-12-10 00:00:00,279.212429,236.41758,199.380533,368.104625,259.380675,125.569898,42.7280,50.9172,71.2261,251.972739,...,NaN,176.906424,137.2758,84.7872,108.4615,NaN,381.538639,26.628133,28.066431,68.832289
